In [34]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OrdinalEncoder

In [35]:
data = pd.read_csv('diabetic_data.csv')

data.describe()
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      101766 non-null  object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    101766 non-null  object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                101766 non-null  object
 11  medical_specialty         101766 non-null  object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [36]:
def print_unique_elements(df, columns):
    for column in columns:
        try:
            print(f"\033[94m{column}\033[0m:")
            unique_counts = df[column].value_counts()
            i = 0
            for value, count in unique_counts.items():
                i = i + 1
                print(f"        \033[91m{value}\033[0m {count}")
        except KeyError:
            print(f"    Column '{column}' not found in the DataFrame.")

In [37]:
print (data.columns)

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')


In [48]:
def print_missing_counts(df):
    for col in df.select_dtypes(include=['object']).columns:
        count = df[col].value_counts().get('?', 0)
        if count > 0:
            print(f"The count of '?' in the column '{col}' is: {count}")

    nas = df.isna().sum()
    nas = nas[nas > 0]
    print(nas)

print_missing_counts(data)

The count of '?' in the column 'race' is: 2273
The count of '?' in the column 'weight' is: 98569
The count of '?' in the column 'payer_code' is: 40256
The count of '?' in the column 'medical_specialty' is: 49949
The count of '?' in the column 'diag_1' is: 21
The count of '?' in the column 'diag_2' is: 358
The count of '?' in the column 'diag_3' is: 1423
max_glu_serum    96420
A1Cresult        84748
dtype: int64


In [43]:
no_missing = data[data['gender'] != 'Unknown/Invalid']
no_missing['gender'].unique()

array(['Female', 'Male'], dtype=object)

this fields have significant number of missing values and dosen't have high importance in data so we can ignore them .

In [44]:
no_missing = no_missing.drop(columns=['max_glu_serum', 'A1Cresult', 'payer_code', 'weight'])

but medical_specialty has high importance in data so we imputed it despite significant number of missing values .

In [45]:
def impute_columns(df):
    encoder = OrdinalEncoder()

    imputer = IterativeImputer(random_state=0)

    columns = df.select_dtypes(include=['object']).apply(lambda x: x.str.contains('\?')).any()

    for column in columns.index[columns]:
        
        print(f'before imputing column {column}')
        
        print_unique_elements(df, [column])
        
        df.loc[:, column] = df[column].replace('?', np.nan)

        df.loc[:, column] = encoder.fit_transform(df[column].values.reshape(-1, 1))

        df.loc[:, column] = imputer.fit_transform(df[column].values.reshape(-1, 1))

        df.loc[:, column] = encoder.inverse_transform(df[column].values.reshape(-1, 1))
        
        print(f'after imputing column {column}')
        
        print_unique_elements(df, [column])

    return df

impute_columns(no_missing)


before imputing column race
race:
        Caucasian 76099
        AfricanAmerican 19210
        ? 2271
        Hispanic 2037
        Other 1505
        Asian 641
after imputing column race
race:
        Caucasian 76099
        AfricanAmerican 19210
        Asian 2912
        Hispanic 2037
        Other 1505
before imputing column medical_specialty
medical_specialty:
        ? 49947
        InternalMedicine 14635
        Emergency/Trauma 7565
        Family/GeneralPractice 7440
        Cardiology 5351
        Surgery-General 3099
        Nephrology 1613
        Orthopedics 1400
        Orthopedics-Reconstructive 1233
        Radiologist 1140
        Pulmonology 871
        Psychiatry 854
        Urology 685
        ObstetricsandGynecology 671
        Surgery-Cardiovascular/Thoracic 652
        Gastroenterology 564
        Surgery-Vascular 533
        Surgery-Neuro 468
        PhysicalMedicineandRehabilitation 391
        Oncology 348
        Pediatrics 254
        Hematology/Oncology 20

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),6,25,1,1,Pediatrics-Endocrinology,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),1,1,7,3,Neurophysiology,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,Neurophysiology,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),1,1,7,2,Neurophysiology,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),1,1,7,1,Neurophysiology,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,443847548,100162476,AfricanAmerican,Male,[70-80),1,3,7,3,Neurophysiology,...,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,443847782,74694222,AfricanAmerican,Female,[80-90),1,4,5,5,Neurophysiology,...,No,Steady,No,No,No,No,No,No,Yes,NO
101763,443854148,41088789,Caucasian,Male,[70-80),1,1,7,1,Neurophysiology,...,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,443857166,31693671,Caucasian,Female,[80-90),2,3,7,10,Surgery-General,...,No,Up,No,No,No,No,No,Ch,Yes,NO


In [49]:
print_missing_counts(no_missing)

Series([], dtype: int64)


In [51]:
def label_encode(df):
    
    le = LabelEncoder()

    object_columns = df.select_dtypes(include=['object']).columns

    for column in object_columns:
        
        df[column] = le.fit_transform(df[column])

        print(f"{column} labels: ", dict(zip(le.classes_, le.transform(le.classes_))))

    return df


In [52]:
label_encode(no_missing)

race labels:  {'AfricanAmerican': 0, 'Asian': 1, 'Caucasian': 2, 'Hispanic': 3, 'Other': 4}
gender labels:  {'Female': 0, 'Male': 1}
age labels:  {'[0-10)': 0, '[10-20)': 1, '[20-30)': 2, '[30-40)': 3, '[40-50)': 4, '[50-60)': 5, '[60-70)': 6, '[70-80)': 7, '[80-90)': 8, '[90-100)': 9}
medical_specialty labels:  {'AllergyandImmunology': 0, 'Anesthesiology': 1, 'Anesthesiology-Pediatric': 2, 'Cardiology': 3, 'Cardiology-Pediatric': 4, 'DCPTEAM': 5, 'Dentistry': 6, 'Dermatology': 7, 'Emergency/Trauma': 8, 'Endocrinology': 9, 'Endocrinology-Metabolism': 10, 'Family/GeneralPractice': 11, 'Gastroenterology': 12, 'Gynecology': 13, 'Hematology': 14, 'Hematology/Oncology': 15, 'Hospitalist': 16, 'InfectiousDiseases': 17, 'InternalMedicine': 18, 'Nephrology': 19, 'Neurology': 20, 'Neurophysiology': 21, 'Obsterics&Gynecology-GynecologicOnco': 22, 'Obstetrics': 23, 'ObstetricsandGynecology': 24, 'Oncology': 25, 'Ophthalmology': 26, 'Orthopedics': 27, 'Orthopedics-Reconstructive': 28, 'Osteopath':

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,2,0,0,6,25,1,1,37,...,0,1,1,0,0,0,0,1,0,2
1,149190,55629189,2,0,1,1,1,7,3,21,...,0,3,1,0,0,0,0,0,1,1
2,64410,86047875,0,0,2,1,1,7,2,21,...,0,1,1,0,0,0,0,1,1,2
3,500364,82442376,2,1,3,1,1,7,2,21,...,0,3,1,0,0,0,0,0,1,2
4,16680,42519267,2,1,4,1,1,7,1,21,...,0,2,1,0,0,0,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,443847548,100162476,0,1,7,1,3,7,3,21,...,0,0,1,0,0,0,0,0,1,1
101762,443847782,74694222,0,0,8,1,4,5,5,21,...,0,2,1,0,0,0,0,1,1,2
101763,443854148,41088789,2,1,7,1,1,7,1,21,...,0,0,1,0,0,0,0,0,1,2
101764,443857166,31693671,2,0,8,2,3,7,10,62,...,0,3,1,0,0,0,0,0,1,2


In [54]:
no_missing.drop_duplicates(subset=None, keep='first', inplace=True)

In [55]:
no_missing.info()

<class 'pandas.core.frame.DataFrame'>
Index: 101763 entries, 0 to 101765
Data columns (total 46 columns):
 #   Column                    Non-Null Count   Dtype
---  ------                    --------------   -----
 0   encounter_id              101763 non-null  int64
 1   patient_nbr               101763 non-null  int64
 2   race                      101763 non-null  int32
 3   gender                    101763 non-null  int32
 4   age                       101763 non-null  int32
 5   admission_type_id         101763 non-null  int64
 6   discharge_disposition_id  101763 non-null  int64
 7   admission_source_id       101763 non-null  int64
 8   time_in_hospital          101763 non-null  int64
 9   medical_specialty         101763 non-null  int32
 10  num_lab_procedures        101763 non-null  int64
 11  num_procedures            101763 non-null  int64
 12  num_medications           101763 non-null  int64
 13  number_outpatient         101763 non-null  int64
 14  number_emergency         

this is an alternative way to impute the missing values for categorical features

no_missing.loc[:, 'race'] = no_missing['race'].replace('?', np.nan)

df_encoded = pd.get_dummies(no_missing)

imputer = KNNImputer(n_neighbors=3)
df_imputed = imputer.fit_transform(df_encoded)

df_imputed = pd.DataFrame(df_imputed, columns = df_encoded.columns)

no_missing.loc[:, 'race'] = df_imputed[['race_Caucasian', 'race_AfricanAmerican', 'race_Hispanic', 'race_Other', 'race_Asian']].idxmax(axis=1).str.replace('race_', '')
mode_value = no_missing['race'].mode()[0]
no_missing.loc[:, 'race'] = no_missing['race'].fillna(mode_value)
print_unique_elements(no_missing , ['race'])

now we try to detect outliers

In [61]:
from sklearn.ensemble import IsolationForest
import pandas as pd
import numpy as np

def detect_outliers(df, n_estimators, contamination):
    # Make a copy of the DataFrame
    df_copy = df.copy()

    # Create the Isolation Forest model
    iso = IsolationForest(n_estimators=n_estimators, contamination=contamination)

    # Fit the model and predict the outliers
    outlier_label = iso.fit_predict(df_copy)

    # Add the outlier labels to the DataFrame copy
    df_copy['outlier'] = outlier_label

    # Separate the outliers into a different DataFrame
    df_outliers = df_copy[df_copy['outlier'] == -1]

    # Remove the 'outlier' column from the DataFrame copy
    df_copy = df_copy[df_copy['outlier'] == 1]
    df_copy.drop('outlier', axis=1, inplace=True)

    # Return the DataFrame without outliers and the DataFrame with only outliers
    return df_copy, df_outliers

# Use the function
df_no_outliers, df_outliers = detect_outliers(no_missing, 50, 0.05)
